# Benchmarking skforecast

This notebook benchmarks the performance (velocity) of the `skforecast` in its different versions and keeps track of the results.

**Notes**

+ In version `0.15.0` the binning of residuals was introduced in multi-series forecasters. This explains the increase in the time taken to fit the model.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
# !pip install --upgrade numpy < 2.0 && \
# pip install --upgrade pandas && \
# pip install --upgrade scikit-learn

In [3]:
# !pip install skforecast==0.15.1
# !pip install skforecast==0.14.0
# !pip install skforecast==0.13.0

In [4]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import sklearn
import joblib
from benchmarking import BenchmarkRunner, plot_benchmark_results
import skforecast
import platform
import psutil
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor

from skforecast.utils import *
if skforecast.__version__ >= '0.14.0':
    from skforecast.recursive import ForecasterRecursive, ForecasterRecursiveMultiSeries
    from skforecast.direct import ForecasterDirect, ForecasterDirectMultiVariate
    from skforecast.model_selection import TimeSeriesFold, backtesting_forecaster, backtesting_forecaster_multiseries
else:
    from skforecast.ForecasterAutoreg import ForecasterAutoreg
    from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
    from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
    from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
    from skforecast.model_selection import backtesting_forecaster, backtesting_forecaster_multiseries

In [5]:
print(f"Python version: {platform.python_version()}")
print(f"skforecast version: {skforecast.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"Computer network name: {platform.node()}")
print(f"Processor type: {platform.processor()}")
print(f"Platform type: {platform.platform()}")
print(f"Operating system: {platform.system()}")
print(f"Operating system release: {platform.release()}")
print(f"Operating system version: {platform.version()}")
print(f"Number of physical cores: {psutil.cpu_count(logical=False)}")
print(f"Number of logical cores: {psutil.cpu_count(logical=True)}")

Python version: 3.11.10
skforecast version: 0.16.0
scikit-learn version: 1.6.1
pandas version: 2.2.3
numpy version: 2.2.5
Computer network name: ITES015-NB0029
Processor type: Intel64 Family 6 Model 141 Stepping 1, GenuineIntel
Platform type: Windows-10-10.0.19045-SP0
Operating system: Windows
Operating system release: 10
Operating system version: 10.0.19045
Number of physical cores: 8
Number of logical cores: 16


In [6]:
import warnings
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="'force_all_finite' was renamed to 'ensure_all_finite'"
)

# ForecasterRecursiveMultiSeries

In [7]:
# Mock data for benchmarking
# ==========================================================
n_series = 600
len_series = 2000
series_dict = {}
rng = np.random.default_rng(321)
for i in range(n_series):
    series_dict[f'series_{i}'] = pd.Series(
        data = rng.normal(loc=20, scale=5, size=len_series),
        index=pd.date_range(
            start='2010-01-01',
            periods=len_series,
            freq='h'
        ),
        name=f'series_{i}'
    )
series_dataframe = pd.DataFrame(series_dict, index=series_dict['series_0'].index)

exog_dict = {}
rng = np.random.default_rng(321)
for k in series_dict.keys():
    exog = pd.DataFrame(
            index=series_dict[k].index
            )
    exog['day_of_week'] = exog.index.dayofweek
    exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
    exog['month'] = exog.index.month
    exog_dict[k] = exog

exog_dict_prediction = {}
for k in series_dict.keys():
    exog = pd.DataFrame(
            index=pd.date_range(
                start=series_dict[k].index.max() + pd.Timedelta(hours=1),
                periods=100,
                freq='h'
            )
            )
    exog['day_of_week'] = exog.index.dayofweek
    exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
    exog['month'] = exog.index.month
    exog_dict_prediction[k] = exog

In [ ]:
# Benchmarking speed ForecasterRecursiveMultiSeries
# ==============================================================================
regressor = LGBMRegressor(random_state=8520, verbose=-1)
# regressor = LinearRegression()
if skforecast.__version__ >= '0.14.0':
    forecaster = ForecasterRecursiveMultiSeries(
        regressor=regressor,
        lags=50,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        encoding="ordinal"
    )
else:
    forecaster = ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=50,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        encoding="ordinal"
    )

def ForecasterRecursiveMultiSeries_fit_series_is_dict_exog_is_dict():
    forecaster.fit(series=series_dict, exog=exog_dict)

def ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dict_exog_is_dict():
    forecaster._create_train_X_y(series=series_dict, exog=exog_dict)

def ForecasterRecursiveMultiSeries_fit_series_is_dataframe_no_exog():
    forecaster.fit(series=series_dataframe)

def ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_no_exog():
    forecaster._create_train_X_y(series=series_dataframe)

def ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_exog_dict():
    forecaster._create_train_X_y(series=series_dataframe, exog=exog_dict)

def ForecasterRecursiveMultiSeries__create_train_X_y_single_series():
    _ = forecaster._create_train_X_y_single_series(
            y = series_dict['series_0'],
            exog = exog_dict['series_0'],
            ignore_exog = False,
        )

def ForecasterRecursiveMultiSeries_predict_exog_is_dict():
    forecaster.predict(steps=100, exog=exog_dict_prediction, suppress_warnings=True)

def ForecasterRecursiveMultiSeries_predict_interval_exog_is_dict_conformal():
    forecaster.predict_interval(
        steps=100,
        exog=exog_dict_prediction,
        method='conformal',
        nterval=[5, 95],
        suppress_warnings=True
    )

def ForecasterRecursiveMultiSeries__create_predict_inputs_exog_is_dict():
    _ = forecaster._create_predict_inputs(
            steps         = 100,
            exog         = exog_dict_prediction,
            check_inputs = True
        )

def ForecasterRecursiveMultiSeries__check_predict_inputs():
    check_predict_input(
        forecaster_name  = type(forecaster).__name__,
        steps            = 100,
        is_fitted        = forecaster.is_fitted,
        exog_in_         = forecaster.exog_in_,
        index_type_      = forecaster.index_type_,
        index_freq_      = forecaster.index_freq_,
        window_size      = forecaster.window_size,
        last_window      = pd.DataFrame(forecaster.last_window_),
        exog             = exog_dict_prediction,
        exog_type_in_    = forecaster.exog_type_in_,
        exog_names_in_   = forecaster.exog_names_in_,
        interval         = None,
        levels           = forecaster.series_names_in_,
        series_names_in_ = forecaster.series_names_in_,
        encoding         = forecaster.encoding
    )

def ForecasterRecursiveMultiSeries_backtesting_series_is_dict_exog_is_dict():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dict,
                exog=exog_dict,
                cv=cv,
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dict,
                exog=exog_dict,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                metric='mean_squared_error',
                show_progress=False
            )
        
def ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_no_exog():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                cv=cv,
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                metric='mean_squared_error',
                show_progress=False
            )
        
def ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_exog_dict():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                exog=exog_dict,
                cv=cv,
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                exog=exog_dict,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                metric='mean_squared_error',
                show_progress=False
            )
        
def ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_exog_dict_conformal():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                exog=exog_dict,
                cv=cv,
                interval=[5, 95],
                interval_method='conformal',
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster_multiseries(
                forecaster=forecaster,
                series=series_dataframe,
                exog=exog_dict,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                interval=[5, 95],
                interval_method='conformal',
                metric='mean_squared_error',
                show_progress=False
            )


runner = BenchmarkRunner(repeat=10, output_dir="./")
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dict_exog_is_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_no_exog, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_exog_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_train_X_y_single_series, forecaster=forecaster)

runner = BenchmarkRunner(repeat=5, output_dir="./")
_ = runner.benchmark(ForecasterRecursiveMultiSeries_fit_series_is_dict_exog_is_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_fit_series_is_dataframe_no_exog, forecaster=forecaster)

forecaster.fit(series=series_dict, exog=exog_dict)
runner = BenchmarkRunner(repeat=10, output_dir="./")
_ = runner.benchmark(ForecasterRecursiveMultiSeries_predict_exog_is_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_predict_interval_exog_is_dict_conformal, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__create_predict_inputs_exog_is_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries__check_predict_inputs, forecaster=forecaster)

runner = BenchmarkRunner(repeat=5, output_dir="./")
_ = runner.benchmark(ForecasterRecursiveMultiSeries_backtesting_series_is_dict_exog_is_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_no_exog, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_exog_dict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursiveMultiSeries_backtesting_series_is_dataframe_exog_dict_conformal, forecaster=forecaster)

Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dict_exog_is_dict with skforecast version 0.15.1
Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_no_exog with skforecast version 0.15.1
Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_series_is_dataframe_exog_dict with skforecast version 0.15.1
Benchmarking function: ForecasterRecursiveMultiSeries__create_train_X_y_single_series with skforecast version 0.15.1
Benchmarking function: ForecasterRecursiveMultiSeries_fit_series_is_dict_exog_is_dict with skforecast version 0.15.1
Benchmarking function: ForecasterRecursiveMultiSeries_fit_series_is_dataframe_no_exog with skforecast version 0.15.1
Benchmarking function: ForecasterRecursiveMultiSeries_predict_exog_is_dict with skforecast version 0.15.1
Benchmarking function: ForecasterRecursiveMultiSeries__create_predict_inputs_exog_is_dict with skforecast version 0.15.1
Benchmarking function: Fore

In [ ]:
# Plot results
# ==============================================================================
display_df = True
selected_date = None
# 'Linux-6.11.0-24-generic-x86_64-with-glibc2.39'
# 'Windows-10-10.0.19045-SP0'
selected_platform = None

results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark = results_benchmark_all.query("forecaster_name in ['ForecasterRecursiveMultiSeries', 'ForecasterAutoregMultiSeries']")
results_benchmark = results_benchmark.query("regressor_name == 'LGBMRegressor'")
for function_name in results_benchmark['function_name'].unique():
    df = results_benchmark.query(f"function_name == '{function_name}'")
    if selected_date:
        df = df[df['datetime'].dt.date == pd.to_datetime(selected_date).date()]
    if selected_platform:
        df = df[df['platform'] == selected_platform]
    if display_df:
        display(df)
    plot_benchmark_results(df.copy(), function_name, add_median=True, add_mean=True)

,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
145,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,8.396984,0.407490,2025-04-24 10:33:15.303767,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
160,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,8.086642,0.110765,2025-04-24 10:40:25.109072,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
175,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,6.684918,0.058411,2025-04-24 11:07:42.003553,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
190,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,8.523445,0.694457,2025-04-24 11:16:37.067503,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
208,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,7.163528,0.064108,2025-04-24 18:03:06.476568,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
228,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,63b606a379874ae9f340ba4fe592d695,6.449687,0.073870,2025-04-24 18:18:13.058177,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
146,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.956854,0.089286,2025-04-24 10:33:25.095433,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
161,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,2.114042,0.050319,2025-04-24 10:40:35.694818,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
176,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.806584,0.027353,2025-04-24 11:07:51.060806,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
191,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.954118,0.050927,2025-04-24 11:16:46.862258,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
205,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.644503,0.049856,2025-04-24 18:02:03.292269,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
224,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,583e7d25f58c2314c85ab6c385d879cc,1.411819,0.062459,2025-04-24 18:17:17.012633,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
147,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,8.617346,0.276325,2025-04-24 10:34:08.214428,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
162,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,6.477069,0.404866,2025-04-24 10:41:08.105175,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
177,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,6.158462,0.055508,2025-04-24 11:08:21.874210,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
192,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,7.924299,0.066108,2025-04-24 11:17:26.506134,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
209,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,6.537754,0.064164,2025-04-24 18:03:39.185807,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
229,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_fit_series_is_d...,aa35b8ba0c0839021a95e5ba0786a67e,5.988162,0.098475,2025-04-24 18:18:43.029344,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
148,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.571599,0.091096,2025-04-24 10:34:16.092443,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
163,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.375553,0.059013,2025-04-24 10:41:15.004316,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
178,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.371918,0.040894,2025-04-24 11:08:28.757259,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
193,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.610562,0.081551,2025-04-24 11:17:34.591231,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
206,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,1.148996,0.047668,2025-04-24 18:02:14.811485,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
225,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,7b994fdfa33ce4a2084e303d3f49c934,0.994326,0.042047,2025-04-24 18:17:26.968773,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
149,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001386,0.000332,2025-04-24 10:34:25.258910,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
164,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.000964,0.000199,2025-04-24 10:41:22.054972,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
179,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.000993,0.000245,2025-04-24 11:08:35.727601,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
194,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.001340,0.000257,2025-04-24 11:17:43.625303,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
210,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.000936,0.000527,2025-04-24 18:03:46.716793,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
227,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ea2ceb1c912245376ba4cea7595175be,0.000717,0.000166,2025-04-24 18:17:40.787470,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07


...

,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
153,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,764622d682985b184e7b2369791c61a1,33.896880,0.406957,2025-04-24 10:37:42.452354,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
168,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,764622d682985b184e7b2369791c61a1,26.872867,0.148170,2025-04-24 10:43:59.665535,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
183,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,764622d682985b184e7b2369791c61a1,27.242620,0.202290,2025-04-24 11:11:15.199890,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
198,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,764622d682985b184e7b2369791c61a1,33.854358,0.383979,2025-04-24 11:21:01.508197,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
214,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,764622d682985b184e7b2369791c61a1,16.077056,0.677136,2025-04-24 18:05:14.429141,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
233,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,764622d682985b184e7b2369791c61a1,14.487273,0.156595,2025-04-24 18:20:08.328570,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
154,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,12.627921,0.484849,2025-04-24 10:38:45.612438,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
169,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,10.287719,0.213035,2025-04-24 10:44:51.118971,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
184,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,10.403703,0.137579,2025-04-24 11:12:07.236935,3.11.10,0.14.0,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
199,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,12.782420,0.155627,2025-04-24 11:22:05.448858,3.11.10,0.15.1,2.1.3,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
215,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,10.698136,0.126667,2025-04-24 18:06:07.947284,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
234,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,77968c38bfc431991b56a6500970e471,9.375887,0.114813,2025-04-24 18:20:55.233667,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
207,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ace3e84de8c6ee0baf7abe80c790a1db,1.580161,0.075582,2025-04-24 18:02:30.636992,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
226,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries__create_train_X...,ace3e84de8c6ee0baf7abe80c790a1db,1.376138,0.041489,2025-04-24 18:17:40.756467,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
216,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,d72f22d955e2149418d02e9e4f8b9a45,15.007106,0.229013,2025-04-24 18:07:23.017057,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
235,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,d72f22d955e2149418d02e9e4f8b9a45,13.279067,0.246881,2025-04-24 18:22:01.650838,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07


,forecaster_name,regressor_name,function_name,function_hash,run_time_avg,run_time_std_dev,datetime,python_version,skforecast_version,numpy_version,pandas_version,sklearn_version,lightgbm_version,platform,processor,cpu_count,memory_gb
217,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,4081578249b9beefbee5821991bb788b,23.776374,0.343084,2025-04-24 18:09:21.934592,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07
236,ForecasterRecursiveMultiSeries,LGBMRegressor,ForecasterRecursiveMultiSeries_backtesting_ser...,4081578249b9beefbee5821991bb788b,25.542821,1.557417,2025-04-24 18:24:09.396025,3.11.10,0.16.0,2.2.5,2.2.3,1.6.1,4.5.0,Windows-10-10.0.19045-SP0,"Intel64 Family 6 Model 141 Stepping 1, Genuine...",16,34.07


# ForecasterRecursive

In [10]:
# Mock data for benchmarking
# ==========================================================
len_series = 2000
rng = np.random.default_rng(321)
y = pd.Series(
        data = rng.normal(loc=20, scale=5, size=len_series),
        index=pd.date_range(
            start='2010-01-01',
            periods=len_series,
            freq='h'
        ),
        name='y'
    )
rng = np.random.default_rng(321)
exog = pd.DataFrame(index=y.index)
exog['day_of_week'] = exog.index.dayofweek
exog['week_of_year'] = exog.index.isocalendar().week.astype(int)
exog['month'] = exog.index.month
exog_prediction = pd.DataFrame(
                    index=pd.date_range(
                        start=series_dict[k].index.max() + pd.Timedelta(hours=1),
                        periods=100,
                        freq='h'
                    )
                 ) 
exog_prediction['day_of_week'] = exog_prediction.index.dayofweek
exog_prediction['week_of_year'] = exog_prediction.index.isocalendar().week.astype(int)
exog_prediction['month'] = exog_prediction.index.month

In [ ]:
# Benchmarking speed ForecasterRecursive
# ==============================================================================
regressor = LGBMRegressor(random_state=8520, verbose=-1)
if skforecast.__version__ >= '0.14.0':
    forecaster = ForecasterRecursive(
        regressor=regressor,
        lags=50,
        transformer_y=StandardScaler(),
        transformer_exog=StandardScaler(),
    )
else:
    forecaster = ForecasterAutoreg(
        regressor=regressor,
        lags=50,
        transformer_y=StandardScaler(),
        transformer_exog=StandardScaler(),
    )

def ForecasterRecursive_fit():
    forecaster.fit(y=y, exog=exog)

def ForecasterRecursive__create_train_X_y():
    if skforecast.__version__ >= '0.14.0':
        forecaster._create_train_X_y(y=y, exog=exog)
    else:
        forecaster.create_train_X_y(y=y, exog=exog)


def ForecasterRecursive_predict():
    forecaster.predict(steps=100, exog=exog_prediction)

def ForecasterRecursive_predict_interval_conformal():
    forecaster.predict(
        steps=100,
        exog=exog_prediction,
        interval=[5, 95],
        interval_method='conformal'
    )


def ForecasterRecursive__create_predict_inputs():
    _ = forecaster._create_predict_inputs(
            steps        = 100,
            exog         = exog_prediction,
            check_inputs = True
        )
    
def ForecasterRecursive_backtesting():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster(
                forecaster=forecaster,
                y=y,
                exog=exog,
                cv=cv,
                metric='mean_squared_error',
                show_progress=False
            )
    else:
        _ = backtesting_forecaster(
                forecaster=forecaster,
                y=y,
                exog=exog,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                metric='mean_squared_error',
                show_progress=False
            )

def ForecasterRecursive_backtesting_conformal():
    if skforecast.__version__ >= '0.14.0':
        cv = TimeSeriesFold(
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
            )
        _ = backtesting_forecaster(
                forecaster=forecaster,
                y=y,
                exog=exog,
                interval=[5, 95],
                interval_method='conformal',
                cv=cv,
                metric='mean_squared_error',

                show_progress=False
            )
    else:
        _ = backtesting_forecaster(
                forecaster=forecaster,
                y=y,
                exog=exog,
                initial_train_size=1200,
                fixed_train_size=True,
                steps=50,
                interval=[5, 95],
                interval_method='conformal',
                metric='mean_squared_error',
                show_progress=False
            )


runner = BenchmarkRunner(repeat=30, output_dir="./")
_ = runner.benchmark(ForecasterRecursive__create_train_X_y, forecaster=forecaster)

runner = BenchmarkRunner(repeat=10, output_dir="./")
_ = runner.benchmark(ForecasterRecursive_fit, forecaster=forecaster)

runner = BenchmarkRunner(repeat=30, output_dir="./")
forecaster.fit(y=y, exog=exog)
_ = runner.benchmark(ForecasterRecursive_predict, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursive_predict_interval_conformal, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursive__create_predict_inputs, forecaster=forecaster)

runner = BenchmarkRunner(repeat=5, output_dir="./")
_ = runner.benchmark(ForecasterRecursive_backtesting, forecaster=forecaster)
_ = runner.benchmark(ForecasterRecursive_backtesting_conformal, forecaster=forecaster)

Benchmarking function: ForecasterRecursive__create_train_X_y with skforecast version 0.15.1
Benchmarking function: ForecasterRecursive_fit with skforecast version 0.15.1
Benchmarking function: ForecasterRecursive_predict with skforecast version 0.15.1
Benchmarking function: ForecasterRecursive__create_predict_inputs with skforecast version 0.15.1
Benchmarking function: ForecasterRecursive_backtesting with skforecast version 0.15.1
Benchmarking function: ForecasterRecursive_backtesting_conformal with skforecast version 0.15.1


In [ ]:
# Plot results
# ==============================================================================
display_df = False
selected_date = None
# 'Linux-6.11.0-24-generic-x86_64-with-glibc2.39'
# 'Windows-10-10.0.19045-SP0'
selected_platform = None

results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark = results_benchmark_all.query("forecaster_name in ['ForecasterRecursive', 'ForecasterAutoreg']")
results_benchmark = results_benchmark.query("regressor_name == 'LGBMRegressor'")
for function_name in results_benchmark['function_name'].unique():
    df = results_benchmark.query(f"function_name == '{function_name}'")
    if selected_date:
        df = df[df['datetime'].dt.date == pd.to_datetime(selected_date).date()]
    if selected_platform:
        df = df[df['platform'] == selected_platform]
    if display_df:
        display(df)
    plot_benchmark_results(df.copy(), function_name, add_median=True, add_mean=True)

# Summary of historical results

In [15]:
results_benchmark_all = joblib.load("./benchmark.joblib")
results_benchmark_all['function_name'] = results_benchmark_all['function_name'].str.split('_', n=1).str[1]
results_benchmark_all = results_benchmark_all.groupby(['forecaster_name', 'skforecast_version', 'function_name'])['run_time_avg'].agg('median').reset_index()
results_benchmark_all = results_benchmark_all.sort_values(by=['function_name'])

fig = px.bar(
    results_benchmark_all.query("forecaster_name in ['ForecasterRecursiveMultiSeries', 'ForecasterAutoregMultiSeries']"),
    x='function_name',
    y='run_time_avg',
    color='skforecast_version',
    barmode='group',
    title='MultiSeries Forecasters - Median Run Time by Function and skforecast Version',
    labels={'run_time_avg': 'Median Run Time (s)', 'function_name': 'Function'},
    category_orders={'skforecast_version': sorted(results_benchmark_all['skforecast_version'].unique())}
)
fig.update_layout(xaxis_tickangle=-45, height=600)
fig.show()

fig = px.bar(
    results_benchmark_all.query("forecaster_name in ['ForecasterRecursive', 'ForecasterAutoreg']"),
    x='function_name',
    y='run_time_avg',
    color='skforecast_version',
    barmode='group',
    title='Single-Series Forecasters - Median Run Time by Function and skforecast Version',
    labels={'run_time_avg': 'Median Run Time (s)', 'function_name': 'Function'},
    category_orders={'skforecast_version': sorted(results_benchmark_all['skforecast_version'].unique())}
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()
